In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<iframe src="https://www.kaggle.com/embed/jiashenliu/introduction-to-financial-concepts-and-data?cellIds=18&kernelSessionId=67183666" height="300" style="margin: 0 auto; width: 100%; max-width: 950px;" frameborder="0" scrolling="auto" title="Introduction to financial concepts and data"></iframe>

In [ ]:
sample = pd.read_csv("../input/optiver-realized-volatility-prediction/sample_submission.csv")
sample

In [ ]:
test = pd.read_csv("../input/optiver-realized-volatility-prediction/test.csv")
test

In [ ]:
book_testparquet = pd.read_parquet("../input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0")
book_testparquet

<iframe src="https://www.kaggle.com/embed/jiashenliu/introduction-to-financial-concepts-and-data?cellIds=18&kernelSessionId=67183666" height="300" style="margin: 0 auto; width: 100%; max-width: 950px;" frameborder="0" scrolling="auto" title="Introduction to financial concepts and data"></iframe>

In [ ]:
trade_testparquet = pd.read_parquet("../input/optiver-realized-volatility-prediction/trade_test.parquet/stock_id=0")
trade_testparquet

In [ ]:
train = pd.read_csv("../input/optiver-realized-volatility-prediction/train.csv")
train

In [ ]:
book_example = pd.read_parquet('../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=0')
book_example


In [ ]:

trade_example = pd.read_parquet("../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=0")
trade_example

In [ ]:
train

In [ ]:
for col in train.columns:
    print(col,':',len(train[col].unique()))
    


In [ ]:
stock = train.groupby('stock_id')['target'].agg(['median','mean','std','count','sum']).reset_index()
stock


In [ ]:
print('mean_value=',stock['mean'].mean())
plt.hist(stock['mean'])

In [ ]:
print('stock_sum=',stock['sum'].mean())
plt.hist(stock['sum'])

In [ ]:
book_example

In [ ]:
book_test = book_example.query('time_id ==5',engine='python')
book_test

In [ ]:
samples = ['bid_price1','bid_price2','ask_price1','ask_price2']
for num,a in enumerate(samples):
    plt.figure(figsize=(20,5))#ウインドウ
    plt.subplot(4,1,num+1)#１行４列分割でグラフを作成、３つ目の引数はサブ領域の番号
    plt.plot(book_test['seconds_in_bucket'],book_test[a])
    plt.title(a)
plt.show()
plt.figure(figsize = (20,5)) 

for num,a in enumerate(samples):
    plt.plot(book_test['seconds_in_bucket'],book_test[a],label=a,)
    plt.legend(fontsize=12)

In [ ]:
trade_test = trade_example.query('time_id==5',engine='python')

plt.figure(figsize=(20,5))

for num,a in enumerate(samples):
    plt.plot(book_test['seconds_in_bucket'],book_test[a],label=a)

plt.plot(trade_test['seconds_in_bucket'],trade_test['price'],label='trade_parquet',lw=10)
plt.legend(fontsize=12)

In [ ]:
trade_test

In [ ]:
stock0 = train.query('stock_id==0',engine='python')
min_index = stock0['target'].idxmin()#最小値の列名を取得
min_time_id = stock0.iloc[min_index]['time_id']#ilocは要素の抽出
print('min_time_id is',min_time_id,'target is',stock0.iloc[min_index]['target'])

In [ ]:
book_test_min = book_example.query('time_id == @min_time_id',engine = 'python')#変数には@をつける
trade_test_min = trade_example.query('time_id== @min_time_id',engine = 'python')

plt.figure(figsize =(20,5))

for num,a in enumerate(samples):
    
    plt.plot(book_test_min['seconds_in_bucket'],book_test_min[a],label=a)

plt.plot(trade_test_min['seconds_in_bucket'],trade_test_min['price'],label='trade_min_parquet',lw=12)
plt.legend(fontsize=12)

In [ ]:
max_index = stock0['target'].idxmax()
max_time_id = stock0.iloc[max_index]['time_id']
print('max_time_id is',max_time_id,'target is',max_index)



In [ ]:
book_test_max =  book_example.query('time_id == @max_time_id',engine = 'python')
trade_test_max = trade_example.query('time_id == @max_time_id',engine = 'python')

plt.figure(figsize=(20,5))

for num,a in enumerate(samples):
    plt.plot(book_test_max['seconds_in_bucket'],book_test_max[a],label=a)

plt.plot(trade_test_max['seconds_in_bucket'],trade_test_max['price'],label = 'trade_max_parquet',lw=12)
plt.legend(fontsize=12)



In [ ]:
plt.figure(figsize=(20,5))
plt.plot(trade_test_min['seconds_in_bucket'],trade_test_min['price'],label = 'trade_min_valatility',lw=12)
plt.plot(trade_test_max['seconds_in_bucket'],trade_test_max['price'],label = 'trade_max_volatility', lw=12)
plt.legend(fontsize=12)

ボラティリティが高いときは価格変動が起きやすい

In [ ]:
stock

In [ ]:
#submit
stock2=stock[['stock_id','median']]
stock2=stock2.set_index('stock_id')
stock2

In [ ]:
#辞書型にする
stock_dict = stock2.to_dict()
#example
stock_dict

In [ ]:
sample

In [ ]:
sample['stock_id'] = [s.split("-")[0] for s in sample['row_id']]
sample

In [ ]:
sample['target'] = [stock_dict['median'][int(s)] for s in sample['stock_id']]#targetに中央値を入れる
sample

In [ ]:
submission = sample.drop('stock_id',axis=1)
submission

In [ ]:
submission.to_csv("submission.csv",index=False)